## Real-time Machine Learning
##### Armin Danesh | 801200833
##### Github Link: https://github.com/adaneshp/Real-Time-Machine-Learning

--------------------

## Problem 1.1:

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [53]:
housing = pd.DataFrame(pd.read_csv("Housing.csv"))
housing.head

<bound method NDFrame.head of         price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0    13300000  7420         4          2        3      yes        no       no   
1    12250000  8960         4          4        4      yes        no       no   
2    12250000  9960         3          2        2      yes        no      yes   
3    12215000  7500         4          2        2      yes        no      yes   
4    11410000  7420         4          1        2      yes       yes      yes   
..        ...   ...       ...        ...      ...      ...       ...      ...   
540   1820000  3000         2          1        1      yes        no      yes   
541   1767150  2400         3          1        1       no        no       no   
542   1750000  3620         2          1        1      yes        no       no   
543   1750000  2910         3          1        1       no        no       no   
544   1750000  3850         3          1        2      yes        no       no  

In [54]:
np.random.seed(0)
df_train, df_test = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state=None)
df_train.shape, df_test.shape

((436, 13), (109, 13))

In [55]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price']
df_Newtrain = df_train[num_vars]
df_Newtest = df_test[num_vars]
df_Newtrain.head()

,area,bedrooms,bathrooms,stories,parking,price
542,3620,2,1,1,0,1750000
496,4000,2,1,1,0,2695000
484,3040,2,1,1,0,2870000
507,3600,2,1,1,0,2590000
252,9860,3,1,1,0,4515000


In [57]:
scaler = MinMaxScaler()
df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])

/tmp/ipykernel_4763/818169181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
/tmp/ipykernel_4763/818169181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])


In [83]:
x_train = torch.tensor(df_Newtrain[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']].values, dtype = torch.float32)
y_train = torch.tensor(df_Newtrain[['price']].values, dtype = torch.float32)
x_test = torch.tensor(df_Newtest[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']].values, dtype = torch.float32)
y_test = torch.tensor(df_Newtest[['price']].values, dtype = torch.float32)


In [84]:
class Net (nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Net, self).__init__()
        self.hidden = nn.Linear(n_inputs, 8)
        self.predict = nn.Linear(8, n_outputs)
    def forward(self, x):
        x = self.hidden(x)
        x = F.tanh(x)
        x = self.predict(x)
        x = F.tanh(x)
        return x


In [113]:
net = Net(5, 1)
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()
optimizer.zero_grad()
for epoch in range(200):
    prediction = net(x_train)
    loss = loss_func(prediction, y_train)
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()
    val_predict = net (x_train)
    val_loss = loss_func(val_predict, y_train)
    print(f"Epoch #{epoch} --> train_loss = {loss}, val_loss = {val_loss}")

Epoch #0 --> train_loss = 0.03600039705634117, val_loss = 0.03594562038779259
Epoch #1 --> train_loss = 0.03594562038779259, val_loss = 0.035891093313694
Epoch #2 --> train_loss = 0.035891093313694, val_loss = 0.035836830735206604
Epoch #3 --> train_loss = 0.035836830735206604, val_loss = 0.0357828326523304
Epoch #4 --> train_loss = 0.0357828326523304, val_loss = 0.03572908788919449
Epoch #5 --> train_loss = 0.03572908788919449, val_loss = 0.03567560017108917
Epoch #6 --> train_loss = 0.03567560017108917, val_loss = 0.03562236949801445
Epoch #7 --> train_loss = 0.03562236949801445, val_loss = 0.035569388419389725
Epoch #8 --> train_loss = 0.035569388419389725, val_loss = 0.0355166532099247
Epoch #9 --> train_loss = 0.0355166532099247, val_loss = 0.035464175045490265
Epoch #10 --> train_loss = 0.035464175045490265, val_loss = 0.03541195020079613
Epoch #11 --> train_loss = 0.03541195020079613, val_loss = 0.03535997495055199
Epoch #12 --> train_loss = 0.03535997495055199, val_loss = 0.035

/home/armin/.local/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


## Problem 1.2:

In [115]:
class Net2 (nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Net2, self).__init__()
        self.hidden1 = nn.Linear(n_inputs, 8)
        self.hidden2 = nn.Linear(8, 4)
        self.hidden3 = nn.Linear(4, 2)
        self.predict = nn.Linear(2, n_outputs)
    def forward(self, x):
        x = self.hidden1(x)
        x = F.tanh(x)
        x = self.hidden2(x)
        x = F.tanh(x)
        x = self.hidden3(x)
        x = F.tanh(x)
        x = self.predict(x)
        x = F.tanh(x)
        return x

In [120]:
net2 = Net2(5, 1)
optimizer = torch.optim.SGD(net2.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()
optimizer.zero_grad()
for epoch in range(200):
    prediction = net2(x_train)
    loss = loss_func(prediction, y_train)
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()
    val_predict = net2 (x_train)
    val_loss = loss_func(val_predict, y_train)
    print(f"Epoch #{epoch} --> train_loss = {loss}, val_loss = {val_loss}")

/home/armin/.local/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch #0 --> train_loss = 0.6256612539291382, val_loss = 0.623363733291626
Epoch #1 --> train_loss = 0.623363733291626, val_loss = 0.6210651397705078
Epoch #2 --> train_loss = 0.6210651397705078, val_loss = 0.6187654733657837
Epoch #3 --> train_loss = 0.6187654733657837, val_loss = 0.6164647936820984
Epoch #4 --> train_loss = 0.6164647936820984, val_loss = 0.6141631603240967
Epoch #5 --> train_loss = 0.6141631603240967, val_loss = 0.6118608117103577
Epoch #6 --> train_loss = 0.6118608117103577, val_loss = 0.609557569026947
Epoch #7 --> train_loss = 0.609557569026947, val_loss = 0.6072534918785095
Epoch #8 --> train_loss = 0.6072534918785095, val_loss = 0.6049487590789795
Epoch #9 --> train_loss = 0.6049487590789795, val_loss = 0.6026434302330017
Epoch #10 --> train_loss = 0.6026434302330017, val_loss = 0.6003375053405762
Epoch #11 --> train_loss = 0.6003375053405762, val_loss = 0.5980311632156372
Epoch #12 --> train_loss = 0.5980311632156372, val_loss = 0.5957242846488953
Epoch #13 -->

In [134]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(net2, (1,5), 
                                           as_strings=True,
                                           print_per_layer_stat=True, 
                                           verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Net2(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (hidden1): Linear(0.0 M, 49.485% Params, 0.0 GMac, 49.485% MACs, in_features=5, out_features=8, bias=True)
  (hidden2): Linear(0.0 M, 37.113% Params, 0.0 GMac, 37.113% MACs, in_features=8, out_features=4, bias=True)
  (hidden3): Linear(0.0 M, 10.309% Params, 0.0 GMac, 10.309% MACs, in_features=4, out_features=2, bias=True)
  (predict): Linear(0.0 M, 3.093% Params, 0.0 GMac, 3.093% MACs, in_features=2, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           97      


In [132]:
macs, params = get_model_complexity_info(net, (1,5), 
                                           as_strings=True,
                                           print_per_layer_stat=True, 
                                           verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Net(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (hidden): Linear(0.0 M, 84.211% Params, 0.0 GMac, 84.211% MACs, in_features=5, out_features=8, bias=True)
  (predict): Linear(0.0 M, 15.789% Params, 0.0 GMac, 15.789% MACs, in_features=8, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           57      


## Problem 2.1:

In [174]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

class_name = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

cifar10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
cifar10_val = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=True)
cifar10, cifar10_val

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ./data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: ./data
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ))

In [175]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(3*32*32, 512)
        self.predict = nn.Linear(512, 10)

    def forward(self, x):
        x = self.hidden(x)
        x = F.tanh(x)
        x = self.predict(x)
        x = F.tanh(x)
        x = F.log_softmax(x, dim=1)
        return x

In [183]:
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = torch.nn.NLLLoss()
optimizer.zero_grad()
for epoch in range(200):
    loss = 0
    val_loss = 0
    total = 0
    correct = 0
    Acc = 0
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = net(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        for imgs, labels in val_loader:
            batch_size = imgs.shape[0]
            outputs = outputs = net(imgs.view(batch_size, -1))
            val_loss = loss_fn(outputs, labels)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
            Acc = (correct / total)*100
    print(f"Epoch #{epoch} --> train_loss: {loss.item():.4f} | val_loss: {val_loss.item():.4f} | Validation Accuracy: {Acc}%")

Epoch #0 --> train_loss: 2.0045 | val_loss: 1.8200 | Validation Accuracy: 36.58%
Epoch #1 --> train_loss: 1.8976 | val_loss: 2.2250 | Validation Accuracy: 37.55%
Epoch #2 --> train_loss: 1.8206 | val_loss: 1.9687 | Validation Accuracy: 38.42%
Epoch #3 --> train_loss: 1.9344 | val_loss: 1.5311 | Validation Accuracy: 38.68%
Epoch #4 --> train_loss: 2.0381 | val_loss: 2.0096 | Validation Accuracy: 38.73%
Epoch #5 --> train_loss: 1.6628 | val_loss: 2.0216 | Validation Accuracy: 39.26%
Epoch #6 --> train_loss: 1.7598 | val_loss: 1.9807 | Validation Accuracy: 39.629999999999995%
Epoch #7 --> train_loss: 1.7964 | val_loss: 1.7895 | Validation Accuracy: 40.26%
Epoch #8 --> train_loss: 1.6592 | val_loss: 1.8183 | Validation Accuracy: 40.52%
Epoch #9 --> train_loss: 1.6561 | val_loss: 1.9971 | Validation Accuracy: 40.03%
Epoch #10 --> train_loss: 1.4846 | val_loss: 1.7445 | Validation Accuracy: 41.31%
Epoch #11 --> train_loss: 2.0398 | val_loss: 1.7089 | Validation Accuracy: 41.4%
Epoch #12 --> 

## Problem 2.2:

In [184]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.hidden1 = nn.Linear(3*32*32, 512)
        self.hidden2 = nn.Linear(512, 128)
        self.hidden3 = nn.Linear(128, 32)
        self.predict = nn.Linear(32, 10)

    def forward(self, x):
        x = self.hidden1(x)
        x = F.tanh(x)
        x = self.hidden2(x)
        x = F.tanh(x)
        x = self.hidden3(x)
        x = F.tanh(x)
        x = self.predict(x)
        x = F.tanh(x)
        x = F.log_softmax(x, dim=1)
        return x

In [185]:
net = Net2()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = torch.nn.NLLLoss()
optimizer.zero_grad()
for epoch in range(200):
    loss = 0
    val_loss = 0
    total = 0
    correct = 0
    Acc = 0
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = net(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        for imgs, labels in val_loader:
            batch_size = imgs.shape[0]
            outputs = outputs = net(imgs.view(batch_size, -1))
            val_loss = loss_fn(outputs, labels)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
            Acc = (correct / total)*100
    print(f"Epoch #{epoch} --> train_loss: {loss.item():.4f} | val_loss: {val_loss.item():.4f} | Validation Accuracy: {Acc}%")

/home/armin/.local/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch #0 --> train_loss: 2.1353 | val_loss: 2.1035 | Validation Accuracy: 31.41%
Epoch #1 --> train_loss: 1.8086 | val_loss: 1.8264 | Validation Accuracy: 33.1%
Epoch #2 --> train_loss: 1.8830 | val_loss: 1.8245 | Validation Accuracy: 34.74%
Epoch #3 --> train_loss: 1.8548 | val_loss: 1.8101 | Validation Accuracy: 36.370000000000005%
Epoch #4 --> train_loss: 2.1736 | val_loss: 1.8715 | Validation Accuracy: 37.46%
Epoch #5 --> train_loss: 2.0371 | val_loss: 1.8636 | Validation Accuracy: 38.45%
Epoch #6 --> train_loss: 1.8180 | val_loss: 1.8129 | Validation Accuracy: 38.769999999999996%
Epoch #7 --> train_loss: 1.8672 | val_loss: 1.9475 | Validation Accuracy: 39.28%
Epoch #8 --> train_loss: 1.6835 | val_loss: 1.6991 | Validation Accuracy: 39.81%
Epoch #9 --> train_loss: 1.9610 | val_loss: 1.5696 | Validation Accuracy: 40.6%
Epoch #10 --> train_loss: 1.8710 | val_loss: 1.6140 | Validation Accuracy: 41.05%
Epoch #11 --> train_loss: 1.8019 | val_loss: 2.0715 | Validation Accuracy: 41.31%
Ep

In [186]:
net_test = Net()
net2_test = Net2()
macs, params = get_model_complexity_info(net_test, (1, 3*32*32), 
                                           as_strings=True,
                                           print_per_layer_stat=True, 
                                           verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))
macs, params = get_model_complexity_info(net2_test, (1, 3*32*32), 
                                           as_strings=True,
                                           print_per_layer_stat=True, 
                                           verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Net(
  1.579 M, 100.000% Params, 0.002 GMac, 100.000% MACs, 
  (hidden): Linear(1.573 M, 99.675% Params, 0.002 GMac, 99.675% MACs, in_features=3072, out_features=512, bias=True)
  (predict): Linear(0.005 M, 0.325% Params, 0.0 GMac, 0.325% MACs, in_features=512, out_features=10, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           1.58 M  
Net2(
  1.643 M, 100.000% Params, 0.002 GMac, 100.000% MACs, 
  (hidden1): Linear(1.573 M, 95.733% Params, 0.002 GMac, 95.733% MACs, in_features=3072, out_features=512, bias=True)
  (hidden2): Linear(0.066 M, 3.995% Params, 0.0 GMac, 3.995% MACs, in_features=512, out_features=128, bias=True)
  (hidden3): Linear(0.004 M, 0.251% Params, 0.0 GMac, 0.251% MACs, in_features=128, out_features=32, bias=True)
  (predict): Linear(0.0 M, 0.020% Params, 0.0 GMac, 0.020% MACs, in_features=32, out_features=10, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           1.64 M  


/home/armin/.local/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
